In [1]:
!pip install -qU langchain langchainhub langchain-community langchain-google-genai langchain-chroma google-genai chromadb langchain-huggingface langchain_nvidia_ai_endpoints langchain-openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.0 MB/s eta 0:00:00
 

In [2]:
import os
import json
from tqdm import tqdm
from copy import deepcopy

from langchain_chroma import Chroma
from langchain.document_loaders import JSONLoader
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors.embeddings_filter import EmbeddingsFilter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [3]:
# Set environment keys
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# gemini_key = user_secrets.get_secret("GEMINI_API_KEY")

# Configuration
retriever_model_name = "models/text-embedding-004"
llm_name = "gemini-2.0-flash"
rerank = True
top_k = 5
chunk_size = 1000
save_file = f"output/{retriever_model_name.replace('/', '_')}{'_rerank' if rerank else ''}_retrieval_test.json"
context_window = 2048
num_output = 256

In [4]:
def rm_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Removed existing file: {file_path}")

rm_file(save_file)

In [5]:
# Load corpus (list of dicts with keys like 'text', 'title', etc.)
from datasets import load_dataset

corpus_data = load_dataset("yixuantt/MultiHopRAG", "corpus")["train"]

README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

corpus.json:   0%|          | 0.00/6.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/609 [00:00<?, ? examples/s]

In [6]:
corpus_data

Dataset({
    features: ['category', 'author', 'published_at', 'body', 'title', 'url', 'source'],
    num_rows: 609
})

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=200
)

# Convert to LangChain Document format and split each doc into chunks
docs = []
for doc in corpus_data:
    metadata = {
        "title": doc.get("title", ""),
        "source": doc.get("source", ""),
        "published_at": doc.get("published_at", ""),
    }
    # Split the text into chunks
    splits = text_splitter.split_text(doc["body"])
    # Create a Document object for each chunk
    for chunk in splits:
        docs.append(Document(page_content=chunk, metadata=metadata))

In [8]:
len(docs)

8448

In [9]:
docs[1].page_content

"We're curating a running list of Amazon's best Cyber Weekend deals, spotlighting some of our favorites and noting when good deals inevitably sell out. Read on for the full rundown, and check back often: We're going to be updating this story incessantly as the sale continues, as well as our even bigger (if you can imagine) list of Cyber Monday deals across more retailers.\n\nNote: All newly added deals are marked with a ✨, while deals with a 🔥 have dropped to an all-time low price. Amazon's invite-only deals for Prime members are marked with a 📨. Deals with a strikeout were either sold out or expired at the time of writing.\n\nEcho, Fire TV, and Kindle deals\n\nWhy we like it"

In [10]:
# embedding_model = GoogleGenerativeAIEmbeddings(model=retriever_model_name, google_api_key=gemini_key)
# Embed and store in Chroma
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vectorstore = Chroma.from_documents(docs, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})





2025-06-01 04:50:38.880584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748753439.086549      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748753439.147595      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_19/1814921782.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model =

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# Create the client
llm = ChatNVIDIA(
    model="google/gemma-2-9b-it",
    api_key="my ass",  # Replace with actual key
    temperature=0.5,
    top_p=1,
    max_tokens=1024,
)

In [13]:
if rerank:
    reranker = EmbeddingsFilter(
        embeddings=embedding_model,
        similarity_threshold=0.5,  # tweak as needed
        k=top_k
    )
    retriever = ContextualCompressionRetriever(
        base_compressor=reranker,
        base_retriever=retriever
    )

In [14]:
from datasets import load_dataset

query_data = load_dataset("yixuantt/MultiHopRAG", "MultiHopRAG")["train"]

MultiHopRAG.json:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2556 [00:00<?, ? examples/s]

In [15]:
query_data

Dataset({
    features: ['evidence_list', 'answer', 'query', 'question_type'],
    num_rows: 2556
})

In [16]:
q = query_data[0]

retrieved_docs = retriever.invoke(q["query"])
    
retrieval_list = []
for doc in retrieved_docs:
    retrieval_list.append({
        "text": doc.page_content,
        "score": doc.metadata.get("relevance_score", 0)  # if available
    })

result = {
    "query": q["query"],
    "answer": q["answer"],
    "question_type": q["question_type"],
    "retrieval_list": retrieval_list,
    "gold_list": q["evidence_list"]
}

result

{'query': 'Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?',
 'answer': 'Sam Bankman-Fried',
 'question_type': 'inference_query',
 'retrieval_list': [{'text': '“In the end, Sam started and built two billion-dollar businesses,” he concluded. “He didn’t steal any money.”\n\nA Frenchman who lives in London testifies in New York\n\nAfter lawyers from both sides depicted two very different Bankman-Frieds, the prosecution called its first two witnesses to the stand—and they weren’t blockbuster names or former lieutenants-turned-government-cooperators, like Ellison.\n\nThe first was a victim: Marc-Antoine Julliard, a Paris-born cocoa trader who lives in London. In 2021, Julliard, who had coiffed hair and spoke with a strong French accent, decided to invest in crypto and landed on FTX as his exchange of cho

In [17]:
# llm = ChatGoogleGenerativeAI(model=llm_name, temperature=0, google_api_key=gemini_key)




In [18]:
# prefix = (
#     "Below is a question followed by some context from different sources. "
#     "Please answer the question based on the context. The answer to the question is a word or entity. "
#     "Answer directly without explanation. The answer MUST in the format ANSWER: <result>. Where <result> is the predicted of the model "
#     "If the provided information is not enough to find the answer of the question, respond 'ANSWER: Insufficient Information.'. "
    
# )

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [

        ("user", "{input}"),
    ]
)

In [20]:
# template = prefix + "\n\nQuestion: {query}\n\nContext:\n\n{context}"
# prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    # print("Got", len(docs), "documents")
    return "\n--------------\n".join([doc.page_content for doc in retrieved_docs])

rag_chain = (
    {"query": RunnablePassthrough(), "context": retriever | format_docs}
    | prompt
    | llm
    | StrOutputParser()
)
chain  = ( prompt | llm | StrOutputParser())

reason_chain  = ( prompt | llm | StrOutputParser())
filter_chain  = ( prompt | llm | StrOutputParser())

In [21]:
len(query_data)

2556

In [22]:
from IPython.display import display, Markdown

q = query_data[2]

print("Answer:", q["answer"])

Answer: Sam Altman


In [23]:
import re
import string
from tqdm import tqdm

def normalize_answer(s):
    """Lower text and remove punctuation, articles, and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punctuation(text):
        return ''.join(ch for ch in text if ch not in string.punctuation)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punctuation(lower(str(s)))))

def exact_match_multiple(prediction, ground_truths):
    """Returns 1 if the prediction matches any ground truth answer exactly, otherwise 0."""
    normalized_prediction = normalize_answer(prediction)
    
    # Ensure ground_truths is a list, even if it's a single string
    if not isinstance(ground_truths, list):
        ground_truths = [ground_truths]

    for truth in ground_truths:
        normalized_truth = normalize_answer(truth)
        if normalized_prediction == normalized_truth:
            return 1
    return 0

def compute_f1_multiple(prediction, ground_truths):
    """Compute F1 score between prediction and multiple ground truths."""
    # Ensure ground_truths is a list
    if not isinstance(ground_truths, list):
        ground_truths = [ground_truths]
    
    # Normalize prediction
    normalized_prediction = normalize_answer(prediction)
    pred_tokens = normalized_prediction.split()
    
    # Find the best F1 score among all ground truths
    best_f1 = 0
    
    for truth in ground_truths:
        normalized_truth = normalize_answer(truth)
        truth_tokens = normalized_truth.split()
        
        # Calculate token overlap
        common_tokens = set(pred_tokens) & set(truth_tokens)
        
        # Calculate precision and recall
        if len(pred_tokens) == 0:
            precision = 0
        else:
            precision = len(common_tokens) / len(pred_tokens)
        
        if len(truth_tokens) == 0:
            recall = 0
        else:
            recall = len(common_tokens) / len(truth_tokens)
        
        # Calculate F1
        if precision + recall == 0:
            f1 = 0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)
        
        # Keep the best F1 score
        best_f1 = max(best_f1, f1)
    
    return best_f1

# Code test RAG Base

In [24]:
# from tqdm import tqdm
# test_count = len(query_data)

# match = 0
# exact_match_count = 0
# total_f1 = 0
# results = []
# for i in tqdm(range(test_count)):
#     # Get prediction from RAG chain
#     res = rag_chain.invoke(query_data[i]["query"])
    
#     # Extract answer from response
#     pred_answer = res.split("ANSWER: ", 1)[1] if len(res.split("ANSWER: ", 1)) >= 2 else ""
#     pred_answer = pred_answer.replace('\n', '')
    
#     if pred_answer == "":
#         print(f"Warning: Empty prediction for query {i}")
#         print(f"Full response: {res}")
#         continue

#     # Get ground truth answer(s)
#     ground_truth = query_data[i]["answer"]
#     if i%200==0:
#         print('pred_answer', pred_answer)
#         print('gt_answer', ground_truth)
#     # Calculate Exact Match
#     em_score = exact_match_multiple(pred_answer, ground_truth)
#     exact_match_count += em_score
    
#     # Calculate F1 Score
#     f1_score = compute_f1_multiple(pred_answer, ground_truth)
#     total_f1 += f1_score
# em_percentage = (exact_match_count / test_count) * 100 if test_count > 0 else 0
# avg_f1_score = total_f1 / test_count if test_count > 0 else 0
# print(em_percentage, avg_f1_score)

# Code test IRCOT

In [25]:
def ircot_pipeline(question, max_steps=5):
    """
    Implements the IRCoT pipeline with interleaved reasoning and retrieval
    """
    query = question
    reasoning_steps = []
    retrieved_contexts = []
    final_answer = ""
    # Initial retrieval based on the question
    initial_nodes = retriever.invoke(query)
    # print(initial_nodes.page_content)
    initial_context = "\n".join([node.page_content for node in initial_nodes])
    # print(initial_context)
    retrieved_contexts.append(initial_context)
    num_step=1
    for step in range(max_steps):
        # Generate reasoning step based on current information
        # Generate the next reasoning step
        # print(prompt)
        prompt ={
            "query": query + f"""Based on the reasoning steps so far and the retrieved information,
        do we have enough evidence to answer the original question?
        MUST ANSWER with'YES' or 'NO'. If 'YES', further answer the question (MUST ANSWER 'YES' BEFORE ANSWER, The format answer is 'YES . ANSWER: <your answer>'  AND DO NOT EXPLAIN THE ANSWER). If 'NO' provide the information need to know to answer the question (MUST ANSWER 'NO' BEFORE ANSWER and ANSWER: <your answer>' with  <your answer> is the information you need to know).""",
            "context":  f"""Retrieved Information: {retrieved_contexts[-1]}
        Previous Reasoning: {' '.join(reasoning_steps) if reasoning_steps else 'None'}"""
        }
        # print(prompt)
        reasoning =chain.invoke(prompt)
        print(reasoning)
        if "NO" not in reasoning and ("YES" in reasoning or 'ANSWER'  in reasoning):
            final_answer = reasoning
            break
        reasoning_steps.append(f"Step {step+1}: {reasoning}")
        # print(f"\nReasoning Step {step+1}: {reasoning}")
        num_step+=1
        # Retrieve based on the latest reasoning
        new_nodes = retriever.invoke(reasoning)
        new_context = "\n".join([node.page_content for node in new_nodes])
        retrieved_contexts.append(new_context)
    # print(final_answer)
    if final_answer =="":
        final_answer="ANSWER: Insufficient Information."
   

    return {
        'answer':final_answer,
        'num_step':num_step
    } 



In [26]:
# from tqdm import tqdm
# test_count = len(query_data)

# match = 0
# exact_match_count = 0
# total_f1 = 0
# results = []
# total_num_iter =0
# for i in tqdm(range(test_count)):
#     # Get prediction from RAG chain
#     result = ircot_pipeline(query_data[i]["query"], max_steps=5)
#     res = result['answer']
#     num_iter = result['num_step']
#     total_num_iter+=num_iter
#     # print(res)
#     # Extract answer from response
#     pred_answer = res.split("ANSWER: ", 1)[1] if len(res.split("ANSWER: ", 1)) >= 2 else ""
#     pred_answer = pred_answer.replace('\n', '')
    
#     if pred_answer == "":
#         # print(f"Warning: Empty prediction for query {i}")
#         # print(f"Full response: {res}")
#         res =  "Insufficient Information."

#     # Get ground truth answer(s)
#     ground_truth = query_data[i]["answer"]
#     # if i%200==0:
#     print('pred_answer', pred_answer)
#     print('gt_answer', ground_truth)
#     # Calculate Exact Match
#     em_score = exact_match_multiple(pred_answer, ground_truth)
#     exact_match_count += em_score
    
#     # Calculate F1 Score
#     f1_score = compute_f1_multiple(pred_answer, ground_truth)
#     total_f1 += f1_score
# em_percentage = (exact_match_count / test_count) * 100 if test_count > 0 else 0
# avg_f1_score = total_f1 / test_count if test_count > 0 else 0
# total_num_iter = total_num_iter / test_count
# print(em_percentage, avg_f1_score, total_num_iter)

# Code test Agent

In [27]:
def ircot_pipeline_agent(question, max_steps=5):
    """
    Implements the IRCoT pipeline with interleaved reasoning and retrieval
    """
    query = question
    reasoning_steps = []
    retrieved_contexts = []
    final_answer = ""
    # Initial retrieval based on the question
    initial_nodes = retriever.invoke(query)
    # print(initial_nodes.page_content)
    initial_context = "\n".join([node.page_content for node in initial_nodes])
    # print(initial_context)
    retrieved_contexts.append(initial_context)
    num_step=0
    
    prompt = {
        "input": query+ """"Carefully evaluate the preceding question. Determine if you possess all the information required to provide a demonstrably accurate, confident, and complete answer.

If you have any doubt whatsoever or identify any missing piece of information (however small) that prevents you from providing a fully comprehensive and confident answer, you MUST select a 'YES' option below.

Respond with exactly one of the following:

NO: I am absolutely certain I have all necessary information to answer the question confidently and completely.
YES ONE: I need exactly one additional piece of information to answer confidently and completely.
YES MULTIPLE: I need multiple additional pieces of information to answer confidently and completely
                    """
    }
    # print(prompt)
    reasoning =chain.invoke(prompt)
    # print('FIRST Reasoning', reasoning)
    if "NO" in reasoning:
        prompt = {
            "input": query + '\n' + f"""Based on the above question, Answer the question to the best of your ability with the information available WITHOUT ANY EXPLANTATION.The answer MUST IN THE FORMAT        
        ANSWER: <your answer>"""
        }
        final_answer = chain.invoke(prompt)
        # print('Answer imeediately', final_answer)
        
    elif "YES ONE"  in reasoning:
        prompt = {
            "input":  query + '\n' + """FROM above question, after planning core component information of the question. YOU MUST RESPOND *EXACTLY* AND *ONLY* IN THE FOLLOWING FORMAT:
REQUIREMENT: [The reasoning step of you answer the question]
""" 
        }
        num_step += 1

        info_need = reason_chain.invoke(prompt)
        # print(info_need)
        info_need = info_need.replace('REQUIREMENT:', '').strip()
        info = retriever.invoke(info_need)
        info = "\n".join([node.page_content for node in info])
        
        prompt = {
            "input":  query + '\n' +'The info retrieved:' +info+ '\n' f"""Based on the above question and the info provided, Answer the question to the best of your ability with the information available WITHOUT ANY EXPLANTATION.The answer MUST IN THE FORMAT        
        ANSWER: <your answer>"""
        }

        final_answer = chain.invoke(prompt)
        
    elif "YES MULTIPLE" in reasoning:
        for step in range(max_steps):
            
            cumulative_context = "\n".join(retrieved_contexts) if retrieved_contexts else ""
            
            prompt = {
                "input":  f"""Original Question: {query}
            
            {f'Information Retrieved So Far:{cumulative_context}' if cumulative_context else ''}
            
            Based on the original question and any information already retrieved, what specific information do you still need to provide a complete answer?
            
            **Instructions:**
            - If you now have sufficient information, respond with: "SUFFICIENT"
            - If you still need more information, specify exactly what you need
            - Focus only on the most critical missing piece for this iteration
            - Be specific about what type of information, data, or details are required
            
            **Response Format:**
            STATUS: [SUFFICIENT | NEED_MORE]
            REQUIREMENT: [Specific information needed, or "None" if sufficient]"""
            }
            reasoning = filter_chain.invoke(prompt)
            reasoning_steps.append(f"Step {step+1}: {reasoning}")
            
            
            # Only retrieve if more information is needed
            if "SUFFICIENT" not in reasoning.upper():
                num_step += 1
                # Extract the specific requirement for retrieval
                requirement_lines = reasoning.split('\n')
                search_query = reasoning  # Default to full reasoning
                
                # Try to extract more focused search query
                for line in requirement_lines:
                    if 'REQUIREMENT:' in line:
                        search_query = line.replace('REQUIREMENT:', '').strip()
                   
                
                # Retrieve based on the specific requirement
                new_nodes = retriever.invoke(search_query)
                new_context = "\n".join([node.page_content for node in new_nodes])
                prompt = {
                "input":  f"""Summarize those information: {new_context} to make it suitable to answer the {query}"""
            }
                new_context = chain.invoke(prompt)
                retrieved_contexts.append(new_context)
                
                # print(f"Step {step+1} - Retrieved context based on: {search_query}")
            else:
                break
           
            prompt = {
            "input": query + '\n' + f"""Based on {' '.join(retrieved_contexts)} and Previous Reasoning: {' '.join(reasoning_steps) if reasoning_steps else 'None'}, provide a direct answer for the question."""
            +
              f"""Based on the above question, Answer the question to the best of your ability with the information available WITHOUT ANY EXPLANTATION.The answer MUST IN THE FORMAT        
    ANSWER: <your answer>.
    
    You can 'ANSWER: Insufficient information.' if the information is not enough to answer the question"""
            }
    
            final_answer = chain.invoke(prompt)
                # print(f"Step {step+1} - Sufficient information gathered")
                
                
        
    # print(final_answer)
    if final_answer =="":
        final_answer="ANSWER: Insufficient Information."
   

    return {
        'answer':final_answer,
        'num_step':num_step
    } 



In [ ]:
def ircot_pipeline_agent(question, max_steps=5):
    """
    Implements the IRCoT pipeline with interleaved reasoning and retrieval
    """
    query = question
    reasoning_steps = []
    retrieved_contexts = []
    final_answer = ""
    # Initial retrieval based on the question
    initial_nodes = retriever.invoke(query)
    # print(initial_nodes.page_content)
    initial_context = "\n".join([node.page_content for node in initial_nodes])
    # print(initial_context)
    retrieved_contexts.append(initial_context)
    num_step=0
    
    prompt = {
        "input": query+ """"Carefully evaluate the preceding question. Determine if you possess all the information required to provide a demonstrably accurate, confident, and complete answer.

If you have any doubt whatsoever or identify any missing piece of information (however small) that prevents you from providing a fully comprehensive and confident answer, you MUST select a 'YES' option below.

Respond with exactly one of the following:

NO: I am absolutely certain I have all necessary information to answer the question confidently and completely.
YES ONE: I need exactly one additional piece of information to answer confidently and completely.
YES MULTIPLE: I need multiple additional pieces of information to answer confidently and completely
                    """
    }
    # print(prompt)
    reasoning =chain.invoke(prompt)
    # print('FIRST Reasoning', reasoning)
    if "NO" in reasoning:
        prompt = {
            "input": query + '\n' + f"""Based on the above question, Answer the question to the best of your ability with the information available WITHOUT ANY EXPLANTATION.The answer MUST IN THE FORMAT        
        ANSWER: <your answer>"""
        }
        final_answer = chain.invoke(prompt)
        # print('Answer imeediately', final_answer)
        
    elif "YES ONE"  in reasoning:
        prompt = {
            "input":  query + '\n' + """FROM above question, after planning core component information of the question. YOU MUST RESPOND *EXACTLY* AND *ONLY* IN THE FOLLOWING FORMAT:
REQUIREMENT: [The reasoning step of you answer the question]
""" 
        }
        num_step += 1

        info_need = reason_chain.invoke(prompt)
        # print(info_need)
        info_need = info_need.replace('REQUIREMENT:', '').strip()
        info = retriever.invoke(info_need)
        info = "\n".join([node.page_content for node in info])
        
        prompt = {
            "input":  query + '\n' +'The info retrieved:' +info+ '\n' f"""Based on the above question and the info provided, Answer the question to the best of your ability with the information available WITHOUT ANY EXPLANTATION.The answer MUST IN THE FORMAT        
        ANSWER: <your answer>"""
        }

        final_answer = chain.invoke(prompt)
        
    elif "YES MULTIPLE" in reasoning:
        for step in range(max_steps):
            
            cumulative_context = "\n".join(retrieved_contexts) if retrieved_contexts else ""
            
            prompt = {
                "input":  f"""Original Question: {query}
            
            {f'Information Retrieved So Far:{cumulative_context}' if cumulative_context else ''}
            
            Based on the original question and any information already retrieved, what specific information do you still need to provide a complete answer?
            
            **Instructions:**
            - If you now have sufficient information, respond with: "SUFFICIENT"
            - If you still need more information, specify exactly what you need
            - Focus only on the most critical missing piece for this iteration
            - Be specific about what type of information, data, or details are required
            
            **Response Format:**
            STATUS: [SUFFICIENT | NEED_MORE]
            REQUIREMENT: [Specific information needed, or "None" if sufficient]"""
            }
            reasoning = filter_chain.invoke(prompt)
            reasoning_steps.append(f"Step {step+1}: {reasoning}")
            
            
            # Only retrieve if more information is needed
            if "SUFFICIENT" not in reasoning.upper():
                num_step += 1
                # Extract the specific requirement for retrieval
                requirement_lines = reasoning.split('\n')
                search_query = reasoning  # Default to full reasoning
                
                # Try to extract more focused search query
                for line in requirement_lines:
                    if 'REQUIREMENT:' in line:
                        search_query = line.replace('REQUIREMENT:', '').strip()
                   
                
                # Retrieve based on the specific requirement
                new_nodes = retriever.invoke(search_query)
                new_context = "\n".join([node.page_content for node in new_nodes])
                prompt = {
                "input":  f"""Summarize those information: {new_context} to make it suitable to answer the {query}"""
            }
                new_context = chain.invoke(prompt)
                retrieved_contexts.append(new_context)
                
                # print(f"Step {step+1} - Retrieved context based on: {search_query}")
            else:
                break
           
            prompt = {
            "input": query + '\n' + f"""Based on {' '.join(retrieved_contexts)} and Previous Reasoning: {' '.join(reasoning_steps) if reasoning_steps else 'None'}, provide a direct answer for the question."""
            +
              f"""Based on the above question, Answer the question to the best of your ability with the information available WITHOUT ANY EXPLANTATION.The answer MUST IN THE FORMAT        
    ANSWER: <your answer>.
    
    You can 'ANSWER: Insufficient information.' if the information is not enough to answer the question"""
            }
    
            final_answer = chain.invoke(prompt)
                # print(f"Step {step+1} - Sufficient information gathered")
                
                
        
    # print(final_answer)
    if final_answer =="":
        final_answer="ANSWER: Insufficient Information."
   

    return {
        'answer':final_answer,
        'num_step':num_step
    } 



In [ ]:
from tqdm import tqdm
test_count = len(query_data)

match = 0
exact_match_count = 0
total_f1 = 0
results = []
total_num_iter =0
for i in tqdm(range(test_count)):
    # Get prediction from RAG chain
    result = ircot_pipeline_agent(query_data[i]["query"], max_steps=5)
    res = result['answer']
    num_iter = result['num_step']
    total_num_iter+=num_iter
    print(res)
    # Extract answer from response
    pred_answer = res.split("ANSWER: ", 1)[1] if len(res.split("ANSWER: ", 1)) >= 2 else ""
    pred_answer = pred_answer.replace('\n', '')
    
    if pred_answer == "":
        # print(f"Warning: Empty prediction for query {i}")
        # print(f"Full response: {res}")
        pred_answer =  "Insufficient Information."

    # Get ground truth answer(s)
    ground_truth = query_data[i]["answer"]
    # if i%200==0:
    print('pred_answer', pred_answer)
    print('gt_answer', ground_truth)
    # Calculate Exact Match
    em_score = exact_match_multiple(pred_answer, ground_truth)
    exact_match_count += em_score
    
    # Calculate F1 Score
    f1_score = compute_f1_multiple(pred_answer, ground_truth)
    total_f1 += f1_score
em_percentage = (exact_match_count / test_count) * 100 if test_count > 0 else 0
avg_f1_score = total_f1 / test_count if test_count > 0 else 0
total_num_iter = total_num_iter / test_count
print(em_percentage, avg_f1_score, total_num_iter)